In [1]:
import keras
import numpy as np
from cbds.deeplearning.models import vgg16

Using TensorFlow backend.


In [2]:
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten


base_model = vgg16(input_shape=(187,187, 3), include_top=False, batch_normalization=False)

layer_index = 15
for layer in base_model.layers[:layer_index]:
    #layer.trainable = True
    layer.trainable = False
for layer in base_model.layers[layer_index:]:
    layer.trainable = True
    
last_conv_layer = base_model.get_layer("block5_pool")
#x = GlobalAveragePooling2D()(last_conv_layer.output)
x = Flatten()(last_conv_layer.output)
x = Dense(512, activation="relu")(x) #, kernel_regularizer=regularizers.l2(1e-4))(x) 
x = Dropout(0.5)(x)
x = Dense(512, activation="relu")(x)         
predictions = Dense(1, activation="sigmoid")(x)
model = Model(base_model.input, predictions)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 187, 187, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 187, 187, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 187, 187, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 93, 93, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 93, 93, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 93, 93, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 46, 46, 128)       0         
__________

# Load high resolution data

Load the high resolution tiles for Heerlen from disk here including the label for each tile. Plot a couple of images to see if they are loaded successfully. Pre-process the images by subtracting the mean (just as in the vgg16 paper). Finally, split the images in a train, validation, and test set.

In [ ]:
from ProjectPaths import ProjectPaths
import os

In [ ]:
image_path = os.path.join(ProjectPaths.instance().image_dir, "hr_2018_18m_all.npy")
labels_path = os.path.join(ProjectPaths.instance().image_dir, "hr_2018_18m_all_labels.npy")

In [3]:
import os
image_path = os.path.join(r"/media/tim/Data/Work/CBS/DeepSolaris/Images", "hr_2018_18m_all.npy")
labels_path = os.path.join(r"/media/tim/Data/Work/CBS/DeepSolaris/Images", "hr_2018_18m_all_labels.npy")

In [4]:
hr_images = np.load(image_path)
labels = np.load(labels_path)

hr_images.shape, labels.shape

((14200, 187, 187, 3), (14200,))

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
_, ax = plt.subplots(5,5, figsize=(16,16))

i = 0
for r in range(5):
    for c in range(5):
        ax[r, c].imshow(hr_images[i][:,:, ::-1])      
        i += 1

In [6]:
hr_images_pre_processed = hr_images / 255

In [7]:
from sklearn.model_selection import train_test_split

train_images, test_validation_images, train_labels, test_validation_labels = train_test_split(hr_images_pre_processed, labels, test_size=0.20)
test_images, validation_images, test_labels, validation_labels = train_test_split(test_validation_images, test_validation_labels, test_size=0.50)

# Train network on HR data

We will train and evaluate the vgg16 network on the high resolution data here. For training we use:
* Image augmentation

In [8]:
from keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    channel_shift_range = 0.1,
    zoom_range = 0.3,
    shear_range = 0.15,                                
    #brightness_range = [0.9, 1.1],
    horizontal_flip=True,
    fill_mode="reflect")

test_datagen = ImageDataGenerator()

In [ ]:
class_weights = {
    0: 1.,
    1: train_labels[train_labels == 0].size / train_labels[train_labels == 1].size 
}
class_weights

In [9]:
from sklearn.utils import class_weight

class_weights = {class_index: weight
                for class_index, weight in enumerate(class_weight.compute_class_weight("balanced",
                                                                                       np.unique(train_labels),
                                                                                       train_labels))}
class_weights

{0: 0.7119578841814991, 1: 1.6794795978710821}

In [18]:
from keras.optimizers import SGD

batch_size = 64
epochs = 20 #100
learning_rate = 0.001
decay_rate = learning_rate / epochs
momentum = 0.9

sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=True)

In [11]:
from keras.optimizers import RMSprop
optimizer = RMSprop()

In [12]:
train_generator = data_generator.flow(train_images, train_labels, batch_size=batch_size)
test_generator = test_datagen.flow(test_images, test_labels, batch_size=batch_size, shuffle=False)

In [19]:
model.compile(sgd, loss="binary_crossentropy", metrics=['accuracy'])

In [20]:
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from datetime import datetime


model_name = "vgg16_full_fc512_fc512_fc1_aug_frozen_rms_prop"

checkpoint_dir = os.path.join(r"/media/tim/Data/Work/CBS/DeepSolaris/", model_name)
if not os.path.exists(checkpoint_dir):
    os.mkdir(checkpoint_dir)

file_in_checkpoint_dir = model_name + ".hdf5"

early_stopping_callback = EarlyStopping(monitor='val_acc', patience=5)
model_checkpoint_callback = ModelCheckpoint(file_in_checkpoint_dir, monitor='val_acc', verbose=True,
                                                save_weights_only=True,
                                                save_best_only=True)

H = model.fit_generator(train_generator,
                    steps_per_epoch=len(train_labels) // batch_size, 
                    epochs=epochs,
                    callbacks=[early_stopping_callback, model_checkpoint_callback],
                    validation_data=test_generator,
                    validation_steps=len(test_labels) // batch_size,
                    class_weight=class_weights)

Epoch 1/20
 24/177 [===>..........................] - ETA: 39s - loss: 7.8403 - acc: 0.3092

KeyboardInterrupt: 

In [ ]:
epochs_ran = len(H.history["loss"])

_, ax = plt.subplots(1,2, figsize=(25, 10))
ax[0].plot(np.arange(0, epochs_ran), H.history["acc"], label="accuracy")
ax[0].plot(np.arange(0, epochs_ran), H.history["val_acc"], label="val_accuracy", color="r")
ax[0].set_xlim([0, epochs_ran])
ax[1].plot(np.arange(0, epochs_ran), H.history["loss"], label="loss", color="g")
ax[1].plot(np.arange(0, epochs_ran), H.history["val_loss"], label="val_loss", color="y")
ax[1].set_xlim([0, epochs_ran])
ax[0].legend()
ax[1].legend()

ax[0].set_ylabel("Loss/Accuracy")
ax[0].set_xlabel("Epochs")

ax[1].set_ylabel("Loss/Accuracy")
ax[1].set_xlabel("Epochs")

In [ ]:
from sklearn.metrics import classification_report

predictions = model.predict(test_images) > 0.5
print(classification_report(test_labels, predictions))

In [ ]:
from sklearn.metrics import classification_report

predictions = model.predict(validation_images) > 0.5
print(classification_report(validation_labels, predictions))

In [ ]:
for layer in model.layers:
    layer.trainable = True
model.compile(sgd, loss="binary_crossentropy", metrics=['accuracy'])

In [ ]:
H = model.fit_generator(train_generator,
                    steps_per_epoch=len(train_labels) // batch_size, 
                    epochs=epochs,
                    callbacks=[early_stopping_callback, model_checkpoint_callback],
                    validation_data=test_generator,
                    validation_steps=len(test_labels) // batch_size,
                    class_weight=class_weights)

In [ ]:
epochs_ran = len(H.history["loss"])

_, ax = plt.subplots(1,2, figsize=(25, 10))
ax[0].plot(np.arange(0, epochs_ran), H.history["acc"], label="accuracy")
ax[0].plot(np.arange(0, epochs_ran), H.history["val_acc"], label="val_accuracy", color="r")
ax[0].set_xlim([0, epochs_ran])
ax[1].plot(np.arange(0, epochs_ran), H.history["loss"], label="loss", color="g")
ax[1].plot(np.arange(0, epochs_ran), H.history["val_loss"], label="val_loss", color="y")
ax[1].set_xlim([0, epochs_ran])
ax[0].legend()
ax[1].legend()

ax[0].set_ylabel("Loss/Accuracy")
ax[0].set_xlabel("Epochs")

ax[1].set_ylabel("Loss/Accuracy")
ax[1].set_xlabel("Epochs")

In [ ]:
from sklearn.metrics import classification_report

predictions = model.predict(test_images) > 0.5
print(classification_report(test_labels, predictions))

In [ ]:
from sklearn.metrics import classification_report

predictions = model.predict(validation_images) > 0.5
print(classification_report(validation_labels, predictions))